In [1]:
%matplotlib inline
%pylab inline
import pylab
pylab.rcParams['figure.figsize'] = (10.0, 8.0)
from tsne import bh_sne
from matplotlib import pyplot as plt
import pandas
import scipy
import numpy as np
import sklearn.manifold
import os
import gc
from time import gmtime, strftime
import seaborn as sns
from os import listdir
from os.path import isfile, join


Populating the interactive namespace from numpy and matplotlib


In [2]:
def load_data(path):
    gc.collect()
    labels = list(pandas.read_hdf(path, '/df').columns.values)#.get('accrued_depr_pct')
    data = pandas.read_hdf(path, '/df')
    np_data = np.array(data).astype(float)
    del(data)
    data=None
    #np_data = np_data / np_data.max(axis=0)
    def scale_linear_by_column(rawpoints, high=1.0, low=0.0):
        mins = np.min(rawpoints, axis=0)
        maxs = np.max(rawpoints, axis=0)
        rng = maxs - mins
        return high - (((high - low) * (maxs - rawpoints)) / rng)
    np_data = scale_linear_by_column(np_data)
    
    np.random.shuffle(np_data)
    np.nan_to_num(np_data)
    
    np_data[np.isnan(np_data)] = 0.0
    
    return np_data, labels


In [3]:
def visualize(muh_data, buckets = 100, show=False, size=40, path=None, bandwidth=2, display_mode='kde', color = None):
    vis_x = muh_data[:, 0]
    vis_y = muh_data[:, 1]


    #histogram definition
    xyrange = [[-size,size],[-size,size]] # data range
    bins = [buckets,buckets] # number of bins
    
    if(show):
        #
        #sns.set(color_codes=True)
        if(display_mode=='kde'):
            sns.kdeplot(vis_x, vis_y, shade=True,  n_levels=20, bw=bandwidth)
        elif display_mode == 'hex':
            sns.jointplot(x=vis_x, y=vis_y, kind="hex", color="k")
        elif display_mode == 'hist':
            hh, locx, locy = scipy.histogram2d(vis_x, vis_y, range=xyrange, bins=bins)
            plt.imshow(np.flipud(hh.T),cmap='jet',extent=np.array(xyrange).flatten(), interpolation='none', shape = (1000,1000))
            plt.colorbar()
        elif display_mode == 'colored':
            #plt.gray()

            plt.scatter(vis_x, vis_y, c=color,  marker='+', facecolor='b', cmap='jet')
            #fig.patch.set_facecolor('blue')

            #plt.set_axis_bgcolor((0, .5, .5))
            #plt.imshow(np.flipud(hh.T),cmap='jet',extent=np.array(xyrange).flatten(), interpolation='none', shape = (1000,1000))
            plt.colorbar()
        plt.show()
    else:
        assert(path != None)
        save_path = folder+"".join(path.split('/')[-2:])
        save_path = save_path+"_buckets:"+str(buckets)
        save_path = save_path  +".png"
        plt.savefig(save_path)
    #plt.clf()

def save(muh_data, folder, tag=""):
    save_path = folder+"np_array_"+tag
    print "saving to",save_path
    np.save(save_path,muh_data)

In [4]:
def process_full():
    paths = [#"/home/isaac/Dropbox/data_for_brian/hcad_features/hcad_df.hd",
             #"/home/isaac/Dropbox/data_for_brian/hcad_features/hcad_df_100.hd",
             #"/home/isaac/Dropbox/data_for_brian/hcad_features/hcad_df_200.hd",
             #"/home/isaac/Dropbox/data_for_brian/hcad_features/hcad_df_400.hd",
              #"/home/isaac/Dropbox/data_for_brian/hcad_features/hcad_df_1000.hd",
              #"/home/isaac/Dropbox/data_for_brian/terrain_features/dsmgrid/terrain_100.hd",
#              "/home/isaac/Dropbox/data_for_brian/terrain_features/dsmgrid/terrain_200.hd",
#              "/home/isaac/Dropbox/data_for_brian/terrain_features/dsmgrid/terrain_400.hd",
#              "/home/isaac/Dropbox/data_for_brian/terrain_features/dsmgrid/terrain_1000.hd",
#               "/home/isaac/Dropbox/data_for_brian/terrain_features/dtmgrid/terrain_100.hd",
#              "/home/isaac/Dropbox/data_for_brian/terrain_features/dtmgrid/terrain_200.hd",
#              "/home/isaac/Dropbox/data_for_brian/terrain_features/dtmgrid/terrain_400.hd",
#              "/home/isaac/Dropbox/data_for_brian/terrain_features/dtmgrid/terrain_1000.hd",
#              "/home/isaac/Dropbox/data_for_brian/wind_features/hcad_interp_withoutpartial_rad100_hist8x8.mat.hd",
#              "/home/isaac/Dropbox/data_for_brian/wind_features/hcad_interp_withoutpartial_rad100_hist16x16.mat.hd",
#              "/home/isaac/Dropbox/data_for_brian/wind_features/hcad_interp_withoutpartial_rad100_hist36x36.mat.hd",
#              "/home/isaac/Dropbox/data_for_brian/wind_features/hcad_interp_withoutpartial_rad200_hist8x8.mat.hd",
#              "/home/isaac/Dropbox/data_for_brian/wind_features/hcad_interp_withoutpartial_rad200_hist16x16.mat.hd",
#              "/home/isaac/Dropbox/data_for_brian/wind_features/hcad_interp_withoutpartial_rad200_hist36x36.mat.hd",
#              "/home/isaac/Dropbox/data_for_brian/wind_features/hcad_interp_withoutpartial_rad400_hist8x8.mat.hd",
#              "/home/isaac/Dropbox/data_for_brian/wind_features/hcad_interp_withoutpartial_rad400_hist16x16.mat.hd",
#              "/home/isaac/Dropbox/data_for_brian/wind_features/hcad_interp_withoutpartial_rad400_hist36x36.mat.hd",
#              "/home/isaac/Dropbox/data_for_brian/wind_features/hcad_interp_withoutpartial_rad1000_hist8x8.mat.hd",
#              "/home/isaac/Dropbox/data_for_brian/wind_features/hcad_interp_withoutpartial_rad1000_hist16x16.mat.hd",
             "/home/isaac/Dropbox/data_for_brian/wind_features/hcad_interp_withoutpartial_rad1000_hist36x36.mat.hd"
            ]

    folder = "./autorun10/"
    if not os.path.exists(folder):
        os.makedirs(folder)
    print "started at "+strftime("%Y-%m-%d %H:%M:%S", gmtime())
    vis_data = None
    for path in paths:
        np_data, column_labels = load_data(path)
        print len(np_data)
        for n in [200000, 400000, None]:
            vis_data = bh_sne(np_data[:n], perplexity=30)
            dataset_name = path.split('/')[-1].split('.')[0]
            save(vis_data, folder, tag = "n="+str(n)+"_of_"+dataset_name)
            for i in range(len(np_data[0,:])):
                print "\n"+"="*20
                print "dataset:", dataset_name
                print "sample size:",len(vis_data)
                print "\ncolumn:",column_labels[i]+":"
                colors = np_data[:n,i]
                print "log plot:"
                visualize(vis_data, show=True, display_mode = 'colored', color = np.log(colors+.1))
                print "linear plot:"
                visualize(vis_data, show=True, display_mode = 'colored', color = colors)

            del(vis_data)
            vis_data=None
            print "done with "+path+" at "+strftime("%Y-%m-%d %H:%M:%S", gmtime())
        del(np_data)
        np_data=None
        gc.collect()
            
process_full()

started at 2015-12-11 06:04:59
Opening /home/isaac/Dropbox/data_for_brian/wind_features/hcad_interp_withoutpartial_rad1000_hist36x36.mat.hd in read-only mode


MemoryError: 

In [ ]:
def visualize_all_data():
    mypath = "/home/isaac/Desktop/devika/autorun10/"
    onlyfiles = [ f for f in listdir(mypath) if isfile(join(mypath,f)) ]

    for file in onlyfiles:
        if file == "np_array_n=None_of_hcad_df.npy":
            load_path = mypath+str(file)#np_array_8000_terrain_100.npy"
            loaded_data = np.load(load_path)
            print "\n\n---->n="+file.split('_')[2] + " " + '_'.join(file.split('_')[3:])
            print "kde bandwidth 1"
            size = max([max(loaded_data[:,0]),max(loaded_data[:,1]),abs(min(loaded_data[:,0])),abs(min(loaded_data[:,1]))])

            visualize(loaded_data, show=True, size=size, bandwidth=.1)
            #print "kde bandwidth 2"
            #visualize(loaded_data, show=True, size=size, bandwidth=2)
            print "hex histogram"
            visualize(loaded_data, show=True, size=size, bandwidth=1, display_mode='hex')
            #print "histogram"
            #visualize(loaded_data, show=True, size=size, bandwidth=1, display_mode='hist')
visualize_all_data()

In [ ]:
def run_tests():
    path = "/home/isaac/Dropbox/data_for_brian/hcad_features/hcad_df.hd"
    np_data, labels = load_data(path)
    for i in range(len(np_data[0])):
        col = np_data[:,i]
        assert len(np_data[0]) == len(labels)
        assert len(col) == 870476
        assert np.max(col) == 1.0
        assert np.min(col) == 0.0
    print "tests pass"
run_tests()